In [44]:
import csv, os, sys
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, StringIndexer
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from nltk.stem.snowball import SnowballStemmer
from pyspark.sql.types import ArrayType, StringType
from time import time
from datetime import timedelta
from sys import argv
import psutil as psu

In [36]:
# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [14]:
spark = init_spark()

# Load data from file into spark

In [15]:
filename_train = "../dataset/train.csv"
filename_test = "../dataset/valid.csv"

train_rdd = spark.read.csv(filename_train, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
test_rdd = spark.read.csv(filename_test, header=True, multiLine=True, inferSchema=True, escape='"', quote='"')
train_rdd.show(5)

+--------+--------------------+--------------------+--------------------+-------------------+--------+
|      Id|               Title|                Body|                Tags|       CreationDate|       Y|
+--------+--------------------+--------------------+--------------------+-------------------+--------+
|34552656|Java: Repeat Task...|<p>I'm already fa...|      <java><repeat>|2016-01-01 00:21:59|LQ_CLOSE|
|34553034|Why are Java Opti...|<p>I'd like to un...|    <java><optional>|2016-01-01 02:03:20|      HQ|
|34553174|Text Overlay Imag...|<p>I am attemptin...|<javascript><imag...|2016-01-01 02:48:24|      HQ|
|34553318|Why ternary opera...|<p>The question i...|<swift><operators...|2016-01-01 03:30:17|      HQ|
|34553755|hide/show fab wit...|<p>I'm using cust...|<android><materia...|2016-01-01 05:21:48|      HQ|
+--------+--------------------+--------------------+--------------------+-------------------+--------+
only showing top 5 rows



In [16]:
training = train_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"])# change to collect()

testing = test_rdd.rdd \
    .map(lambda x: (x["Title"]+" "+x["Body"], x["Y"])) \
    .toDF(["Question", "Output"]) # change to collect()

# Prepare pipeline

In [47]:
@timer
def get_stop_word_remover(input_col_name, stopwords):
    return StopWordsRemover(inputCol=input_col_name, outputCol="filtered").setStopWords(stopwords)
    

In [48]:
# HEURISTIC 1 - Tokenize the words
regexTokenizer = RegexTokenizer(inputCol="Question", outputCol="words", pattern="\\W")

# HEURISTIC 2 - Remove the stopwords
# Reading stopwords file and storing each word in a list by specifying the return to line delimiter
stop_words = []
text_file = open("../dataset/stop_words.txt", "r")
stop_words = text_file.read().split('\n')
add_stopwords = ["the", "a", "be", "of", "and", "to", "why"] 
stopwordsRemover = get_stop_word_remover("words", stop_words)


BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 6.6%
Uptime: 19:18:48.718848
Disk in use: 65.5%
Memory in use: 6.674312591552734GiB
Dsik free: 5.499122619628906GiB
Time on CPU: 7:35:15.750000
AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 5.1%
Uptime: 19:18:48.898850
Disk in use: 65.5%
Memory in use: 6.673244476318359GiB
Dsik free: 5.499122619628906GiB
Time on CPU: 7:35:16
Execution Time: 0.07918906211853027


In [53]:
@timer
def get_bag_of_word_model(features_col_name, label_col_name):
    countVectors = CountVectorizer(inputCol=features_col_name, outputCol="features")
    indexed_features = StringIndexer(inputCol = label_col_name, outputCol = "label")
    return countVectors, indexed_features
    
    

In [54]:
countVectors_h1, indexed_features_h1 = get_bag_of_word_model("words", "Output")
countVectors_h2, indexed_features_h2 = get_bag_of_word_model("filtered", "Output")

BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 7.7%
Uptime: 19:21:47.528509
Disk in use: 65.5%
Memory in use: 6.625556945800781GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:47.660760
Disk in use: 65.5%
Memory in use: 6.624839782714844GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000
Execution Time: 0.03131103515625
BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:47.761858
Disk in use: 65.5%
Memory in use: 6.624351501464844GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:47.871834
Disk in use: 65.5%
Memory in use: 6.623928070068359GiB
Dsik free: 5.49658203125GiB
Time on CPU: 7:36:48.240000
Execution Time: 0.009263992309570312


In [55]:
@timer
def get_pipeline(*args):
    return Pipeline(stages=[*args])

In [56]:
pipeline_h1 = get_pipeline(regexTokenizer, countVectors_h1, indexed_features_h1)
pipeline_h2 = get_pipeline(regexTokenizer, stopwordsRemover, countVectors_h2, indexed_features_h2)
# pipeline_h3 = get_pipeline(regexTokenizer, stopwordsRemover, countVectors_h2, indexed_features_h2,stemmer_udf)

BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 6.3%
Uptime: 19:21:52.424083
Disk in use: 65.5%
Memory in use: 6.653079986572266GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.890000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 4.9%
Uptime: 19:21:52.527166
Disk in use: 65.5%
Memory in use: 6.653156280517578GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.940000
Execution Time: 0.0004260540008544922
BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 4.8%
Uptime: 19:21:52.628059
Disk in use: 65.5%
Memory in use: 6.653175354003906GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.980000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:21:52.734210
Disk in use: 65.5%
Memory in use: 6.653194427490234GiB
Dsik free: 5.496364593505859GiB
Time on CPU: 7:36:51.980000
Execution Time: 0.0003941059112548828


# Process data through Pipeline train & test

In [57]:
@timer
def get_pipeline_model(pipeline, data):
    """ We should use the same pipeline model on training and testing """
    return pipeline.fit(data)


In [21]:
# model_pipeline = pipeline_h1.fit(training)
model_pipeline = get_pipeline_model(pipeline_h1, training)
data_h1_train  = model_pipeline.transform(training) #what does the transform do?
data_h1_train.show()

Time: 16.91487216949463
+--------------------+--------+--------------------+--------------------+-----+
|            Question|  Output|               words|            features|label|
+--------------------+--------+--------------------+--------------------+-----+
|Java: Repeat Task...|LQ_CLOSE|[java, repeat, ta...|(201488,[0,1,2,3,...|  1.0|
|Why are Java Opti...|      HQ|[why, are, java, ...|(201488,[0,1,4,7,...|  0.0|
|Text Overlay Imag...|      HQ|[text, overlay, i...|(201488,[0,1,2,3,...|  0.0|
|Why ternary opera...|      HQ|[why, ternary, op...|(201488,[0,1,2,3,...|  0.0|
|hide/show fab wit...|      HQ|[hide, show, fab,...|(201488,[0,1,4,5,...|  0.0|
|Accessing pointer...|LQ_CLOSE|[accessing, point...|(201488,[0,1,2,3,...|  1.0|
|How To Disable 2n...| LQ_EDIT|[how, to, disable...|(201488,[1,4,8,15...|  2.0|
|Resizing containe...| LQ_EDIT|[resizing, contai...|(201488,[1,2,3,4,...|  2.0|
|Changing Theme in...|      HQ|[changing, theme,...|(201488,[0,1,2,3,...|  0.0|
|TextBox Value D

**Vocabulary**

In [61]:
# vocabulary frequency without using the countVectorizer helper column that we generated
counts = data_h1_train.select(f.explode('words').alias('col')).groupBy('col').count().take(20)
print({row['col']: row['count'] for row in counts})

{'1293400': 1, 'mypass': 5, 'connected': 433, 'few': 902, 'input': 9663, 'online': 599, '389999': 1, 'travel': 74, '836400': 1, 'those': 1212, 'still': 2194, 'thread1': 26, 'hope': 513, 'recognize': 120, 'parentheses': 89, 'arguments': 696, 'persist': 74, '2bxhsys2c47eyjfhpmroalpxz5suigeubqu7hjuvfvwpoa0xri3iljvhq5qgbwtwpe1x0': 2, 'pabu': 1, 'some': 8532}


In [62]:
# using the countVectorizer helper column that we generated
# https://stackoverflow.com/questions/50255356/pyspark-countvectorizer-and-word-frequency-in-a-corpus
counts = data_h1_train.select('words').take(20)
print(dict(zip(vocabulary, counts[0]['words'].values)))

NameError: name 'model' is not defined

In [73]:
data_h1_test = process_data_in_pipeline(pipeline_h1, testing)
data_h1_test.show()

+--------------------+--------+--------------------+--------------------+-----+
|            Question|  Output|               words|            features|label|
+--------------------+--------+--------------------+--------------------+-----+
|How to get all th...| LQ_EDIT|[how, to, get, al...|(86590,[1,2,4,6,9...|  2.0|
|Retrieve all exce...| LQ_EDIT|[retrieve, all, e...|(86590,[1,2,7,9,1...|  2.0|
|Pandas: read_html...|      HQ|[pandas, read_htm...|(86590,[0,1,2,3,4...|  0.0|
|Reader Always gim...| LQ_EDIT|[reader, always, ...|(86590,[1,2,4,5,7...|  2.0|
|php rearrange arr...| LQ_EDIT|[php, rearrange, ...|(86590,[1,2,4,5,6...|  2.0|
|How do I make a c...|LQ_CLOSE|[how, do, i, make...|(86590,[0,1,2,3,4...|  1.0|
|how can i create ...| LQ_EDIT|[how, can, i, cre...|(86590,[1,5,6,9,1...|  2.0|
|Re-exporting ES6 ...|      HQ|[re, exporting, e...|(86590,[0,1,2,3,4...|  0.0|
|Fetch API with Co...|      HQ|[fetch, api, with...|(86590,[0,1,2,4,5...|  0.0|
|Print list conten...|LQ_CLOSE|[print, l

In [13]:
data_h1_test  = model_pipeline.transform(testing)
data_h1_test.show()

+--------------------+--------+--------------------+--------------------+-----+
|            Question|  Output|               words|            features|label|
+--------------------+--------+--------------------+--------------------+-----+
|How to get all th...| LQ_EDIT|[how, to, get, al...|(201488,[1,2,4,6,...|  2.0|
|Retrieve all exce...| LQ_EDIT|[retrieve, all, e...|(201488,[1,2,7,8,...|  2.0|
|Pandas: read_html...|      HQ|[pandas, read_htm...|(201488,[0,1,2,3,...|  0.0|
|Reader Always gim...| LQ_EDIT|[reader, always, ...|(201488,[1,2,4,5,...|  2.0|
|php rearrange arr...| LQ_EDIT|[php, rearrange, ...|(201488,[1,2,4,5,...|  2.0|
|How do I make a c...|LQ_CLOSE|[how, do, i, make...|(201488,[0,1,2,3,...|  1.0|
|how can i create ...| LQ_EDIT|[how, can, i, cre...|(201488,[1,5,6,8,...|  2.0|
|Re-exporting ES6 ...|      HQ|[re, exporting, e...|(201488,[0,1,2,3,...|  0.0|
|Fetch API with Co...|      HQ|[fetch, api, with...|(201488,[0,1,2,4,...|  0.0|
|Print list conten...|LQ_CLOSE|[print, l

# Split datasets

In [58]:
@timer
def split_dataset(data, distribution):
    return data.randomSplit([distribution, 1-distribution], seed = 1234)
    

In [15]:
train_h1, validate_h1 = split_dataset(data_h1_train, 0.7)

In [76]:
# test1,test2 = split_dataset(data_h1_test, 0.7) # just for testing purposes

# Hyperparameter tuning | is this doing anything? we are not using any data

In [59]:
@timer
def get_best_smoothing_values(target_col, prediction_col):
    # Create grid to find best smoothing
    nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
    paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()
#     cvEvaluator = BinaryClassificationEvaluator(rawPredictionCol=prediction_col)
    cvEvaluator = MulticlassClassificationEvaluator(labelCol=target_col, predictionCol=prediction_col)

    # Cross-validate all smoothing values
    cv = CrossValidator(estimator=nb, estimatorParamMaps=paramGrid, evaluator=cvEvaluator)
    return cv
    

In [17]:
cv = get_best_smoothing_values("label", "prediction")

# Train Model

In [18]:
cvModel = cv.fit(train_h1)

# Predict

In [19]:
# Make predictions on validation set
cvPredictions = cvModel.transform(validate_h1)

In [20]:
type(cvPredictions)

pyspark.sql.dataframe.DataFrame

In [21]:
cvPredictions.select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  2.0|       1.0|
|  1.0|       1.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  1.0|       1.0|
|  0.0|       1.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [22]:
cvPredictions_test = cvModel.transform(data_h1_test)

In [23]:
cvPredictions_test.select("label","prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  2.0|       2.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  2.0|       2.0|
|  2.0|       2.0|
|  1.0|       1.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  2.0|       1.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  2.0|       2.0|
|  0.0|       0.0|
|  1.0|       0.0|
|  2.0|       2.0|
|  2.0|       2.0|
+-----+----------+
only showing top 20 rows



# Evaluate Performance of model

In [60]:
@timer
def evaluate_model(target_col, prediction_col, predictionAndTarget):
#     evaluator = BinaryClassificationEvaluator(rawPredictionCol=prediction_col)
    evaluatorMulti = MulticlassClassificationEvaluator(labelCol=target_col, predictionCol=prediction_col)
#     accuracy = evaluatorMulti.evaluate(predictions)
    # Get metrics
    acc = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "accuracy"})
    f1 = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "f1"})
    weightedPrecision = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedPrecision"})
    weightedRecall = evaluatorMulti.evaluate(predictionAndTarget, {evaluatorMulti.metricName: "weightedRecall"})
#     auc = evaluator.evaluate(predictionAndTarget)
    print ("Model Accuracy: ", acc)
    print ("Model f1-score: ", f1)
    print ("Model weightedPrecision: ", weightedPrecision)
    print ("Model weightedRecall: ", weightedRecall)
#     return accuracy

In [26]:
# Validation Dataset
evaluate_model("label", "prediction", cvPredictions.select("label","prediction"))

Model Accuracy:  0.7653008490217793
Model f1-score:  0.7675841714238029
Model weightedPrecision:  0.7901628512971111
Model weightedRecall:  0.7653008490217792


In [27]:
# testing dataset
evaluate_model("label", "prediction", cvPredictions_test.select("label","prediction"))

Model Accuracy:  0.7633333333333333
Model f1-score:  0.7649182102284617
Model weightedPrecision:  0.7866094786891012
Model weightedRecall:  0.7633333333333333


# Creating one big iteration for all the pipelines

In [25]:
%%time
for pipeline, pipe_name in zip([pipeline_h1, pipeline_h2],["Pipeline 1","Pipeline 2"]):
    print("#######"+ pipe_name + "#######")
    print("**"+ "Creating pipeline model" + "**") 
    model_pipeline = get_pipeline_model(pipeline, training)
    print("**"+ "Transforming training data" + "**") 
    data_train  = model_pipeline.transform(training)
    print("**"+ "Transforming testing data" + "**") 
    data_test  = model_pipeline.transform(testing)
    print("**"+ "Hyperparameter tuning" + "**") 
    cv = get_best_smoothing_values("label", "prediction")
    print("**"+ "Training the model" + "**") 
    cvModel = cv.fit(data_train)
    print("**"+ "predicting on testing dataset" + "**") 
    cvPredictions = cvModel.transform(data_test)
    print("**"+ "Measuring performance" + "**")
    evaluate_model("label", "prediction", cvPredictions.select("label","prediction"))

#######Pipeline 1#######
**Creating pipeline model**
Time: 16.767117977142334
**Transforming training data**
**Transforming testing data**
**Hyperparameter tuning**
Time: 0.03666400909423828
**Training the model**
**predicting on testing dataset**
**Measuring performance**
Model Accuracy:  0.7734666666666666
Model f1-score:  0.7755420672257562
Model weightedPrecision:  0.7947480066182155
Model weightedRecall:  0.7734666666666666
Time: 28.150495052337646
#######Pipeline 2#######
**Creating pipeline model**
Time: 17.788862943649292
**Transforming training data**
**Transforming testing data**
**Hyperparameter tuning**
Time: 0.011992931365966797
**Training the model**
**predicting on testing dataset**
**Measuring performance**
Model Accuracy:  0.7816
Model f1-score:  0.7840113017879425
Model weightedPrecision:  0.7998494840023584
Model weightedRecall:  0.7816
Time: 33.55508613586426
CPU times: user 1.29 s, sys: 238 ms, total: 1.53 s
Wall time: 4min 35s


In [61]:
%%time
for pipeline, pipe_name in zip([pipeline_h1, pipeline_h2],["Pipeline 1","Pipeline 2 with new stop words"]):
    print("#######"+ pipe_name + "#######")
    print("**"+ "Creating pipeline model" + "**") 
    model_pipeline = get_pipeline_model(pipeline, training)
    print("**"+ "Transforming training data" + "**") 
    data_train  = model_pipeline.transform(training)
    print("**"+ "Transforming testing data" + "**") 
    data_test  = model_pipeline.transform(testing)
    print("**"+ "Hyperparameter tuning" + "**") 
    cv = get_best_smoothing_values("label", "prediction")
    print("**"+ "Training the model" + "**") 
    cvModel = cv.fit(data_train)
    print("**"+ "predicting on testing dataset" + "**") 
    cvPredictions = cvModel.transform(data_test)
    print("**"+ "Measuring performance" + "**")
    evaluate_model("label", "prediction", cvPredictions.select("label","prediction"))

#######Pipeline 1#######
**Creating pipeline model**
BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 6.3%
Uptime: 19:22:16.616642
Disk in use: 65.5%
Memory in use: 6.592823028564453GiB
Dsik free: 5.492912292480469GiB
Time on CPU: 7:37:10.150000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 7.3%
Uptime: 19:22:35.311528
Disk in use: 65.5%
Memory in use: 6.654502868652344GiB
Dsik free: 5.490562438964844GiB
Time on CPU: 7:37:53.890000
Execution Time: 18.59303617477417
**Transforming training data**
**Transforming testing data**
**Hyperparameter tuning**
BEFORE CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 4.9%
Uptime: 19:22:35.705788
Disk in use: 65.5%
Memory in use: 6.679847717285156GiB
Dsik free: 5.490562438964844GiB
Time on CPU: 7:37:54.660000


 AFTER CALL TO FUNCTION


 SYSTEMINFO

CPU in use: 0.0%
Uptime: 19:22:35.849007
Disk in use: 65.5%
Memory in use: 6.686481475830078GiB
Dsik free: 5.490562438964844GiB
Time on CPU: 7:37:54.660000
Execution Time: 0.0417780876159668
**Train

# Performance Analysis

In [50]:
def timer(fun):
    
    def wrapper(*args, **kwargs):
        print('BEFORE CALL TO FUNCTION')
        system_info()
        start = time()
        rv = fun(*args, **kwargs)
        duration = time() - start
        print('\n\n AFTER CALL TO FUNCTION')
        system_info()
        print("Execution Time: {}".format(duration))
        return rv
    
    return wrapper

In [63]:
def system_info():
    info = {
        "Uptime": timedelta(seconds=time()-psu.boot_time()),
        "CPU in use": "{}%".format(psu.cpu_percent(interval=.1)),
        "Time on CPU": timedelta(seconds=psu.cpu_times().system+psu.cpu_times().user),
        "Memory in use": "{}GiB".format(psu.virtual_memory().available/(1024**3)),
        "Disk in use": "{}%".format(psu.disk_usage('/').percent),
        "Disk free": "{}GiB".format(psu.disk_usage('/').free/(1024**3)),
        
    }
    
    print("\n\n SYSTEMINFO\n\n" + "\n".join(["{}: {}".format(key,value) for key,value in info.items()]))

In [26]:
system_info()



 SYSTEMINFO

CPU in use: 15.0%
Uptime: 19:15:26.625819
Disk in use: 65.5%
Memory in use: 6.625919342041016MiB
Dsik free: 5.499961853027344MiB
Time on CPU: 7:32:57.350000


In [25]:
print(timedelta(seconds=time()-psu.boot_time()))

19:15:19.796094


In [24]:
seconds=time()-psu.boot_time()

In [62]:
import multiprocessing
multiprocessing.cpu_count()

8